In [ ]:
#pip install llama_index PyPDF2
#pip install llama-index
#pip install llama-index-extractors-entity

In [ ]:
# Function to load a PDF file and extract text
from llama_index.core import Document
from llama_index.core.schema import MetadataMode
from llama_index.llms.openai import OpenAI
import os

def load_pdf(file_path):
    doc = fitz.open(file_path)
    text = ""
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        text += page.get_text()
    return text
# Define the path to your PDF file
pdf_path = "/Users/pradhikshasuresh/Documents/Python/The Solar System.pdf"  
pdf_text = load_pdf(pdf_path)

document_metadata = {
    "file_name": "The Solar System.pdf",  
    "category": "Science",
    "author": "Pradhiksha"
}
document = Document(
    text=pdf_text,
    metadata=document_metadata
)
documents = [document]

#setup OpenAPI key
os.environ["OPENAI_API_KEY"] = "My_openapi_key"
llm = OpenAI(model="gpt-3.5-turbo", temperature=0.2)

from llama_index.core.extractors import (
    TitleExtractor,
    QuestionsAnsweredExtractor,
    SummaryExtractor,
    TitleExtractor,
    KeywordExtractor
)
from llama_index.core.node_parser import SentenceSplitter
#Define Extractors
entity_extractor = EntityExtractor(
    prediction_threshold=0.5,
    label_entities=False,  
    device="cpu",  
)
qa_extractor = QuestionsAnsweredExtractor(questions=3,llm=llm)
summary_extractor = SummaryExtractor(summaries=["prev", "self"], llm=llm)
title_extractor = TitleExtractor(nodes=5, llm=llm)
keyword_extractor = KeywordExtractor(keywords=10, llm=llm)
node_parser = SentenceSplitter()

transformations = [node_parser, title_extractor, entity_extractor,summary_extractor, qa_extractor, keyword_extractor]

from llama_index.core.ingestion import IngestionPipeline
pipeline = IngestionPipeline(transformations=transformations)

nodes = pipeline.run(documents=documents)

#print metadata
import random
samples = random.sample(nodes, 1)
for node in samples:
    print(node.metadata)